# Industrial economy

## Aim of work : 
* Compare classic Cournot oligopoly equilibrium with results obtained with a simple iterative process where players evolve their optimal strategies
* Different market concentration [monopoly, duopoly, triopoly]
* One sided auction : 
    1. Each indiv. firm selects its outpout
    2. Simultaneous offer of quantities to the market
    3. Compute a uniform price for all of these quantities
    4. Each indiv. compute its profits
    5. Thanks to profit each indiv. submits new quanity
> Repeat

### Market rules
$$P(Q) = 600 - Q $$
$$Q = \sum q_i $$
* All players are homogeneous hence : cost = 0

### Production design
* Random pick between 
*With n : number of firms*
$$Q \in ]0;5*IntegePart(120/n]$$
* Produce between 0 and a max capacity 
* All firms have identical production
* Set of individual bid is discrete : 
    * Can only take values such as 5-units increments : 
    * We have a bid index link to a quantity produced
    * 5,10 ... K-5, K with K=5
    * Set of S 
    
    